In [1]:
from datasets import load_dataset

dataset_id = "w11wo/FourSquare-NYC-POI"
dataset = load_dataset(dataset_id, split="train")

/srv/scratch/CRUISE/Wilson/micromamba/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset[0]

{'system_prompt': 'You are user 0 and your basic information is as follows:\nAge: adult; Gender: male; Education: college & beyond; SocioEco: middle.\nYou have the following traits: extroverted, agreeable, conscientious, emotionally stable, open.\nYou have the following preferences: socializing at bars, enjoying diverse cuisines, visiting coffee shops, exploring flea markets, watching movies.\nYou have the following routines: frequent visits to bars and restaurants, regular coffee outings, attending social events on weekends.\nUser 0 is an outgoing adult male, likely in his late 20s to early 30s, who enjoys an active social life. He frequently visits bars, and restaurants, and is open to trying various cuisines, as indicated by his choices of American, French, and burger joints. His preference for social settings suggests a high level of extroversion and agreeability. User 0 also appears conscientious, as he maintains a routine that involves balancing social activities, including atten

In [3]:
def create_reasoning_prompt(datum):
    system_prompt = datum["system_prompt"]
    trajectory = datum["inputs"]
    target = datum["targets"]
    cot_prompt = """With a step-by-step reasoning, suggest why they might have intended on visiting that POI. Analyze the user's profiles, previous visits in trajectory, time of visit of the POI, and their routines. DO NOT mention or analyze the selected POI id at all. You may suggest POI categories that they might have wanted to visit at that hour and based on their profile and trajectory. Return your response in JSON format: {{"profile_analysis": ..., "trajectory_analysis": ..., "time_of_visit_analysis": ..., "routines_and_preferences_analysis": ..., "pontential_categories_of_interest_analysis": ..., "verdict": ...}}"""
    prompt = f"{system_prompt}\n\n{trajectory}\n\n{target}\n\n{cot_prompt}"
    return prompt

In [4]:
import openai

openai.api_key_path = "/import/glass/1/z5601796/.openai_key"

response = openai.ChatCompletion.create(
    model="gpt-4o-mini-2024-07-18",
    response_format={"type": "json_object"},
    messages=[
        {
            "role": "user",
            "content": create_reasoning_prompt(dataset[0]),
        }
    ],
)

result = response.choices[0].message.content

In [5]:
import json

json.loads(result)

{'profile_analysis': 'User 0 is an extroverted, sociable adult male likely in his late 20s to early 30s, who enjoys diverse cultural experiences and social gatherings. His traits, such as being agreeable and open, suggest that he values social connections and is keen to explore various environments.',
 'trajectory_analysis': "The previous visits on June 10th to a flea market, beer garden, and hot dog joint indicate user 0's enjoyment of social, lively settings and quick dining options. These choices reflect his propensity to engage in casual, communal environments where he can enjoy food and socialize.",
 'time_of_visit_analysis': "The planned visit at 14:05:49 suggests a preference for an afternoon outing. Typically, this time would be suitable for a light meal, coffee, or a relaxed social setting, aligning with user 0's established routines of frequent cafe or bar visits during daytime hours.",
 'routines_and_preferences_analysis': 'User 0 has a routine that includes regular visits t